[![在 Colab 中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-3/time-travel.ipynb) [![在 LangChain Academy 中打开](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58239536-lesson-5-time-travel)


# 时间旅行

## 回顾

我们讨论了 human-in-the-loop 的动机：

(1) `Approval`（批准）- 我们可以中断智能体，向用户展示状态，并允许用户接受某个动作

(2) `Debugging`（调试）- 我们可以回滚图，以复现或规避问题

(3) `Editing`（编辑）- 你可以修改状态 

我们展示了断点如何在特定节点停止图，或者让图自己动态中断。

随后我们演示了如何通过人类批准继续执行，或借助人类反馈直接编辑图状态。

## 目标

现在，我们来看看 LangGraph 如何通过查看、重放甚至从过去状态分叉来[支持调试](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/time-travel/)。 

我们称之为 `time travel`（时间旅行）。


In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langgraph_sdk langgraph-prebuilt

In [ ]:
import os, getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


# _set_env("OPENAI_API_KEY")
_set_env("DASHSCOPE_API_KEY")

让我们构建这个智能体。


In [ ]:
# from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatTongyi


def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


def divide(a: int, b: int) -> float:
    """Divide a by b.

    Args:
        a: first int
        b: second int
    """
    return a / b


tools = [add, multiply, divide]
# llm = ChatOpenAI(model="gpt-4o")
llm = ChatTongyi(model="qwen-plus")
llm_with_tools = llm.bind_tools(tools)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [5]:
from IPython.display import Image, display

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# System message
sys_msg = SystemMessage(
    content="You are a helpful assistant tasked with performing arithmetic on a set of inputs."
)


# Node
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}


# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine the control flow
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = builder.compile(checkpointer=MemorySaver())

# Show
# display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

像之前一样运行它。


In [6]:
# Input
initial_input = {"messages": HumanMessage(content="Multiply 2 and 3")}

# Thread
thread = {"configurable": {"thread_id": "1"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_1ba5b4f72e0c4010b63462)
 Call ID: call_1ba5b4f72e0c4010b63462
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6
================================== Ai Message ==================================

The result of multiplying 2 and 3 is 6.


## 浏览历史

我们可以使用 `get_state`，根据 `thread_id` 查看图的**当前**状态！


In [7]:
graph.get_state({"configurable": {"thread_id": "1"}})

StateSnapshot(values={'messages': [HumanMessage(content='Multiply 2 and 3', additional_kwargs={}, response_metadata={}, id='58ed5667-a394-420e-8588-6de9bf0bf278'), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_1ba5b4f72e0c4010b63462', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 2, "b": 3}'}}]}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': 'cc7d7ce7-0b68-4098-b537-b4bb1d25291d', 'token_usage': {'input_tokens': 349, 'output_tokens': 24, 'total_tokens': 373, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--22977c02-57bd-4bb6-bfec-7d8d474caca0-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_1ba5b4f72e0c4010b63462', 'type': 'tool_call'}]), ToolMessage(content='6', name='multiply', id='e44cbe9a-6b7c-430a-9df3-0207b360a1af', tool_call_id='call_1ba5b4f72e0c4010b63462'), AIMessage(content='The result of multiplying 2 and 3 is 6.', additional_kwargs={}

我们还可以浏览智能体的状态历史。

`get_state_history` 让我们获取所有先前步骤的状态。


In [8]:
all_states = [s for s in graph.get_state_history(thread)]

In [9]:
len(all_states)

5

第一个元素就是当前状态，就像我们通过 `get_state` 得到的那样。


In [10]:
all_states[-2]

StateSnapshot(values={'messages': [HumanMessage(content='Multiply 2 and 3', additional_kwargs={}, response_metadata={}, id='58ed5667-a394-420e-8588-6de9bf0bf278')]}, next=('assistant',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09aefd-45c7-62da-8000-02b009dd63f3'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-26T15:45:29.917500+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09aefd-45c4-63b4-bfff-fc767cbc4688'}}, tasks=(PregelTask(id='da882728-621a-5dcd-e674-4e3cf44d3082', name='assistant', path=('__pregel_pull', 'assistant'), error=None, interrupts=(), state=None, result={'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_1ba5b4f72e0c4010b63462', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 2, "b": 3}'}}]}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_

以上这些我们都可以在这里可视化： 

![fig1.jpg](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbb038211b544898570be3_time-travel1.png)


## 重放 

我们可以从任何先前的步骤重新运行智能体。

![fig2.jpg](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbb038a0bd34b541c78fb8_time-travel2.png)


回头看看接收人类输入的那个步骤吧！


In [11]:
to_replay = all_states[-2]

In [12]:
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Multiply 2 and 3', additional_kwargs={}, response_metadata={}, id='58ed5667-a394-420e-8588-6de9bf0bf278')]}, next=('assistant',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09aefd-45c7-62da-8000-02b009dd63f3'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-26T15:45:29.917500+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09aefd-45c4-63b4-bfff-fc767cbc4688'}}, tasks=(PregelTask(id='da882728-621a-5dcd-e674-4e3cf44d3082', name='assistant', path=('__pregel_pull', 'assistant'), error=None, interrupts=(), state=None, result={'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_1ba5b4f72e0c4010b63462', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 2, "b": 3}'}}]}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_

查看一下当时的状态。


In [13]:
to_replay.values

{'messages': [HumanMessage(content='Multiply 2 and 3', additional_kwargs={}, response_metadata={}, id='58ed5667-a394-420e-8588-6de9bf0bf278')]}

我们可以看到接下来要调用的节点。


In [14]:
to_replay.next

('assistant',)

我们还会得到 config，它告诉我们 `checkpoint_id` 和 `thread_id`。


In [15]:
to_replay.config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09aefd-45c7-62da-8000-02b009dd63f3'}}

要从这里重放，只需把这个 config 再传给智能体！

图知道这个检查点已经执行过了。 

它会直接从这个检查点重放！


In [ ]:
############### different from last lesson #####################
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    event["messages"][-1].pretty_print()
############### different from last lesson #####################

================================ Human Message =================================

Multiply 2 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_fbb411a1878246ffa32d66)
 Call ID: call_fbb411a1878246ffa32d66
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6
================================== Ai Message ==================================

The result of multiplying 2 and 3 is 6.


现在，我们可以看到智能体重新运行后的当前状态。


## 分叉

如果我们想从同一步骤开始，但使用不同的输入怎么办。

这就是分叉。

![fig3.jpg](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbb038f89f2d847ee5c336_time-travel3.png)


In [17]:
to_fork = all_states[-2]
to_fork.values["messages"]

[HumanMessage(content='Multiply 2 and 3', additional_kwargs={}, response_metadata={}, id='58ed5667-a394-420e-8588-6de9bf0bf278')]

同样，我们拥有 config。


In [18]:
to_fork.config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09aefd-45c7-62da-8000-02b009dd63f3'}}

让我们修改这个检查点的状态。

只需在调用 `update_state` 时提供 `checkpoint_id` 即可。 

回想一下 `messages` 上的归约器是如何工作的： 

* 如果不提供消息 ID，它会追加。
* 提供消息 ID 时，就会覆盖该消息而不是追加到状态中！

因此，要覆盖这条消息，我们只需要提供它的 ID，也就是 `to_fork.values["messages"].id`。


In [ ]:
############### different from last lesson #####################
fork_config = graph.update_state(
    to_fork.config,
    {
        "messages": [
            HumanMessage(
                content="Multiply 5 and 3", id=to_fork.values["messages"][0].id
            )
        ]
    },
)
############### different from last lesson #####################

In [20]:
fork_config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09af01-08db-6d04-8001-2b34e112cc6c'}}

这会创建一个全新的分叉检查点。
 
但元数据——例如下一步该去哪——依然保留！ 

我们可以看到智能体的当前状态已经用我们的分叉更新了。


In [21]:
all_states = [state for state in graph.get_state_history(thread)]
all_states[0].values["messages"]

[HumanMessage(content='Multiply 5 and 3', additional_kwargs={}, response_metadata={}, id='58ed5667-a394-420e-8588-6de9bf0bf278')]

In [22]:
graph.get_state({"configurable": {"thread_id": "1"}})

StateSnapshot(values={'messages': [HumanMessage(content='Multiply 5 and 3', additional_kwargs={}, response_metadata={}, id='58ed5667-a394-420e-8588-6de9bf0bf278')]}, next=('assistant',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09af01-08db-6d04-8001-2b34e112cc6c'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-09-26T15:47:10.903821+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09aefd-45c7-62da-8000-02b009dd63f3'}}, tasks=(PregelTask(id='db06eb17-a84a-28ad-2cfb-0c561cba7143', name='assistant', path=('__pregel_pull', 'assistant'), error=None, interrupts=(), state=None, result=None),), interrupts=())

现在，当我们进行流式传输时，图知道这个检查点从未执行过。

因此，图会运行，而不是只是重放。


In [23]:
for event in graph.stream(None, fork_config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Multiply 5 and 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_c7b943060059459a938337)
 Call ID: call_c7b943060059459a938337
  Args:
    a: 5
    b: 3
================================= Tool Message =================================
Name: multiply

15
================================== Ai Message ==================================

The result of multiplying 5 and 3 is 15.


现在我们可以看到当前状态是智能体运行结束后的样子。


In [24]:
graph.get_state({"configurable": {"thread_id": "1"}})

StateSnapshot(values={'messages': [HumanMessage(content='Multiply 5 and 3', additional_kwargs={}, response_metadata={}, id='58ed5667-a394-420e-8588-6de9bf0bf278'), AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_c7b943060059459a938337', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 5, "b": 3}'}}]}, response_metadata={'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '211a064d-c567-4f89-9ea6-2c14bad75a22', 'token_usage': {'input_tokens': 349, 'output_tokens': 24, 'total_tokens': 373, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--5a9fc1c5-7d52-4402-8186-3fdd99dbf223-0', tool_calls=[{'name': 'multiply', 'args': {'a': 5, 'b': 3}, 'id': 'call_c7b943060059459a938337', 'type': 'tool_call'}]), ToolMessage(content='15', name='multiply', id='962210aa-081c-4be4-a1fd-d59f344693c4', tool_call_id='call_c7b943060059459a938337'), AIMessage(content='The result of multiplying 5 and 3 is 15.', additional_kwargs=

### 结合 LangGraph API 的时间旅行

**⚠️ 免责声明**

自从录制这些视频以来，我们已经更新了 Studio，使其可以在本地运行并在浏览器中打开。现在推荐的方式是以这种形式运行 Studio（而不是像视频中展示的桌面应用）。关于本地开发服务器请查看[这里](https://langchain-ai.github.io/langgraph/concepts/langgraph_studio/#local-development-server)的文档，关于本地 Studio 的运行方式请查看[这里](https://langchain-ai.github.io/langgraph/how-tos/local-studio/#run-the-development-server)。在本模块的 `/studio` 目录中，在终端运行以下命令即可启动本地开发服务器：

```
langgraph dev
```

你应该会看到如下输出：
```
- 🚀 API: http://127.0.0.1:2024
- 🎨 Studio UI: https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024
- 📚 API Docs: http://127.0.0.1:2024/docs
```

在浏览器中访问 Studio UI：`https://smith.langchain.com/studio/?baseUrl=http://127.0.0.1:2024`。

我们通过 SDK 与之连接，并演示 LangGraph API 如何[支持时间旅行](https://langchain-ai.github.io/langgraph/cloud/how-tos/human_in_the_loop_time_travel/#initial-invocation)。


In [ ]:
if "google.colab" in str(get_ipython()):
    raise Exception(
        "Unfortunately LangGraph Studio is currently not supported on Google Colab"
    )

In [25]:
from langgraph_sdk import get_client

client = get_client(url="http://127.0.0.1:2024")

#### 重放 

让我们运行智能体，并在每个节点执行后流式传回 `updates`。


In [26]:
initial_input = {"messages": HumanMessage(content="Multiply 2 and 3")}
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input=initial_input,
    stream_mode="updates",
):
    if chunk.data:
        assisant_node = chunk.data.get("assistant", {}).get("messages", [])
        tool_node = chunk.data.get("tools", {}).get("messages", [])
        if assisant_node:
            print("-" * 20 + "Assistant Node" + "-" * 20)
            print(assisant_node[-1])
        elif tool_node:
            print("-" * 20 + "Tools Node" + "-" * 20)
            print(tool_node[-1])

--------------------Assistant Node--------------------
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_37525ff8ddf747f0a46d2d', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 2, "b": 3}'}}]}, 'response_metadata': {'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '99bca50f-c458-423a-b8d2-1122ff469a6d', 'token_usage': {'input_tokens': 350, 'output_tokens': 24, 'total_tokens': 374, 'prompt_tokens_details': {'cached_tokens': 0}}}, 'type': 'ai', 'name': None, 'id': 'run--6362de73-5865-4ce7-ab4c-364e2817705b-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_37525ff8ddf747f0a46d2d', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
--------------------Tools Node--------------------
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': 'ec28a17c-2a57-4e75-bef8-51a2bad39fa5', 'tool_call_id': 'ca

接下来看看如何从指定的检查点进行**重放**。 

我们只需要传入 `checkpoint_id`。


In [27]:
states = await client.threads.get_history(thread["thread_id"])
to_replay = states[-2]
to_replay

{'values': {'messages': [{'content': 'Multiply 2 and 3',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': 'd1114cc0-e3d8-408c-a978-807250eb9542',
    'example': False}]},
 'next': ['assistant'],
 'tasks': [{'id': 'fbd2ef0a-4851-6376-9f51-f968befebf4f',
   'name': 'assistant',
   'path': ['__pregel_pull', 'assistant'],
   'error': None,
   'interrupts': [],
   'checkpoint': None,
   'state': None,
   'result': {'messages': [{'content': '',
      'additional_kwargs': {'tool_calls': [{'index': 0,
         'id': 'call_37525ff8ddf747f0a46d2d',
         'type': 'function',
         'function': {'name': 'multiply', 'arguments': '{"a": 2, "b": 3}'}}]},
      'response_metadata': {'model_name': 'qwen-plus',
       'finish_reason': 'tool_calls',
       'request_id': '99bca50f-c458-423a-b8d2-1122ff469a6d',
       'token_usage': {'input_tokens': 350,
        'output_tokens': 24,
        'total_tokens': 374,
        'prompt_tokens_details': 

我们使用 `stream_mode="values"` 来重放，这样就能在每个节点看到完整状态。


In [28]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input=None,
    stream_mode="values",
    checkpoint_id=to_replay["checkpoint_id"],
):
    print(f"Receiving new event of type: {chunk.event}...")
    print(chunk.data)
    print("\n\n")

Receiving new event of type: metadata...
{'run_id': '019986b5-9fbf-71d8-abcd-c43bc001a344', 'attempt': 1}



Receiving new event of type: values...
{'messages': [{'content': 'Multiply 2 and 3', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'd1114cc0-e3d8-408c-a978-807250eb9542', 'example': False}]}



Receiving new event of type: values...
{'messages': [{'content': 'Multiply 2 and 3', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'd1114cc0-e3d8-408c-a978-807250eb9542', 'example': False}, {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_cd98c112d8f34791935767', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 2, "b": 3}'}}]}, 'response_metadata': {'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '05e4ba1d-3ad5-473c-9041-db3b435a4dd9', 'token_usage': {'input_tokens': 350, 'output_tokens': 24, 'total_tokens': 374, 'prompt_tokens_deta

我们也可以把它视作只流式返回被重放节点对状态所做的 `updates`。


In [29]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input=None,
    stream_mode="updates",
    checkpoint_id=to_replay["checkpoint_id"],
):
    if chunk.data:
        assisant_node = chunk.data.get("assistant", {}).get("messages", [])
        tool_node = chunk.data.get("tools", {}).get("messages", [])
        if assisant_node:
            print("-" * 20 + "Assistant Node" + "-" * 20)
            print(assisant_node[-1])
        elif tool_node:
            print("-" * 20 + "Tools Node" + "-" * 20)
            print(tool_node[-1])

--------------------Assistant Node--------------------
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_f81fbb887bcc4ac2beb48e', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 2, "b": 3}'}}]}, 'response_metadata': {'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '7bdf1906-8c62-425c-a016-fe318dd1f53f', 'token_usage': {'input_tokens': 350, 'output_tokens': 24, 'total_tokens': 374, 'prompt_tokens_details': {'cached_tokens': 256}}}, 'type': 'ai', 'name': None, 'id': 'run--2615f05d-5de0-45e1-90df-8cce156de776-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_f81fbb887bcc4ac2beb48e', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
--------------------Tools Node--------------------
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '40828985-7d3e-463a-897c-fe99dfa20f6e', 'tool_call_id': '

#### 分叉

现在看看分叉。

我们获取与之前相同的步骤，也就是人类输入那一步。

我们用智能体创建一个新的线程。


In [30]:
initial_input = {"messages": HumanMessage(content="Multiply 2 and 3")}
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input=initial_input,
    stream_mode="updates",
):
    if chunk.data:
        assisant_node = chunk.data.get("assistant", {}).get("messages", [])
        tool_node = chunk.data.get("tools", {}).get("messages", [])
        if assisant_node:
            print("-" * 20 + "Assistant Node" + "-" * 20)
            print(assisant_node[-1])
        elif tool_node:
            print("-" * 20 + "Tools Node" + "-" * 20)
            print(tool_node[-1])

--------------------Assistant Node--------------------
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_a75ddfb7e3be4991a8ffa1', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 2, "b": 3}'}}]}, 'response_metadata': {'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': '8b47cd21-0d73-43f5-94ae-731747c834b5', 'token_usage': {'input_tokens': 350, 'output_tokens': 24, 'total_tokens': 374, 'prompt_tokens_details': {'cached_tokens': 0}}}, 'type': 'ai', 'name': None, 'id': 'run--f0f31dfe-bd6f-48b5-a216-2efb0553cd02-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_a75ddfb7e3be4991a8ffa1', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
--------------------Tools Node--------------------
{'content': '6', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '91685157-02ac-4524-bd27-58ad34aeb99b', 'tool_call_id': 'ca

In [31]:
states = await client.threads.get_history(thread["thread_id"])
to_fork = states[-2]
to_fork["values"]

{'messages': [{'content': 'Multiply 2 and 3',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': '7db23df6-f432-4446-ab14-5c44028070c0',
   'example': False}]}

In [32]:
to_fork["values"]["messages"][0]["id"]

'7db23df6-f432-4446-ab14-5c44028070c0'

In [33]:
to_fork["next"]

['assistant']

In [34]:
to_fork["checkpoint_id"]

'1f09af03-7b11-68d6-8000-4ef900dfe6a2'

我们来编辑状态。

回想一下 `messages` 上的归约器是如何工作的： 

* 如果不提供消息 ID，它会追加。
* 提供消息 ID 时，就会覆盖该消息而不是追加到状态中！


In [ ]:
############### different from last lesson #####################
forked_input = {
    "messages": HumanMessage(
        content="Multiply 3 and 3", id=to_fork["values"]["messages"][0]["id"]
    )
}

forked_config = await client.threads.update_state(
    thread["thread_id"], forked_input, checkpoint_id=to_fork["checkpoint_id"]
)
############### different from last lesson #####################

In [36]:
forked_config

{'checkpoint': {'thread_id': '5ea829a5-02b9-482d-b08d-ad5bab7ad205',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09af04-5e7b-6360-8001-6392ad2f0d50'},
 'configurable': {'thread_id': '5ea829a5-02b9-482d-b08d-ad5bab7ad205',
  'checkpoint_ns': '',
  'checkpoint_id': '1f09af04-5e7b-6360-8001-6392ad2f0d50'},
 'checkpoint_id': '1f09af04-5e7b-6360-8001-6392ad2f0d50'}

In [37]:
states = await client.threads.get_history(thread["thread_id"])
states[0]

{'values': {'messages': [{'content': 'Multiply 3 and 3',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '7db23df6-f432-4446-ab14-5c44028070c0',
    'example': False}]},
 'next': ['assistant'],
 'tasks': [{'id': '3b7e3c4f-9dca-00ea-af9d-ed5d1d7c6553',
   'name': 'assistant',
   'path': ['__pregel_pull', 'assistant'],
   'error': None,
   'interrupts': [],
   'checkpoint': None,
   'state': None,
   'result': None}],
 'metadata': {'graph_id': 'agent',
  'thread_id': '5ea829a5-02b9-482d-b08d-ad5bab7ad205',
  'checkpoint_id': '1f09af03-7b11-68d6-8000-4ef900dfe6a2',
  'checkpoint_ns': '',
  'source': 'update',
  'step': 1,
  'parents': {}},
 'created_at': '2025-09-26T15:48:40.412642+00:00',
 'checkpoint': {'checkpoint_id': '1f09af04-5e7b-6360-8001-6392ad2f0d50',
  'thread_id': '5ea829a5-02b9-482d-b08d-ad5bab7ad205',
  'checkpoint_ns': ''},
 'parent_checkpoint': {'checkpoint_id': '1f09af03-7b11-68d6-8000-4ef900dfe6a2',
  'thread_id'

要重新运行，只需传入 `checkpoint_id`。


In [ ]:
async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input=None,
    stream_mode="updates",
    ############### different from last lesson #####################
    checkpoint_id=forked_config["checkpoint_id"],
    ############### different from last lesson #####################
):
    if chunk.data:
        assisant_node = chunk.data.get("assistant", {}).get("messages", [])
        tool_node = chunk.data.get("tools", {}).get("messages", [])
        if assisant_node:
            print("-" * 20 + "Assistant Node" + "-" * 20)
            print(assisant_node[-1])
        elif tool_node:
            print("-" * 20 + "Tools Node" + "-" * 20)
            print(tool_node[-1])

--------------------Assistant Node--------------------
{'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_f355c59b54b4470c8c09ed', 'type': 'function', 'function': {'name': 'multiply', 'arguments': '{"a": 3, "b": 3}'}}]}, 'response_metadata': {'model_name': 'qwen-plus', 'finish_reason': 'tool_calls', 'request_id': 'd3dd415e-1261-4f2d-89ce-4aee2be93cd6', 'token_usage': {'input_tokens': 350, 'output_tokens': 24, 'total_tokens': 374, 'prompt_tokens_details': {'cached_tokens': 256}}}, 'type': 'ai', 'name': None, 'id': 'run--0e4ffc10-4c70-4790-bed5-c91d349d8f17-0', 'example': False, 'tool_calls': [{'name': 'multiply', 'args': {'a': 3, 'b': 3}, 'id': 'call_f355c59b54b4470c8c09ed', 'type': 'tool_call'}], 'invalid_tool_calls': [], 'usage_metadata': None}
--------------------Tools Node--------------------
{'content': '9', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'tool', 'name': 'multiply', 'id': '5aa0bc31-f765-48d0-bdd1-17cdb305f3b6', 'tool_call_id': '

### LangGraph Studio

我们在 Studio UI 中查看这个智能体的分叉，它使用的是 `module-1/studio/langgraph.json` 指定的 `module-1/studio/agent.py`。
